## This notebook is about the search of a list of MRN number in another list

### Import the dependencies

In [1]:
import pandas as pd
import sqlite3

### Connect to the database

In [2]:
conn = sqlite3.connect('list.db')
cur = conn.cursor()

%reload_ext sql
%sql sqlite:///list.db

### Import the files as a dataframe

In [7]:
request = pd.read_excel('plan.xlsx', sheet_name='first')
request.to_sql('plan', conn, if_exists='replace', index=False)
conn.commit()
#print(request.head(5))

export = pd.read_excel('export_list.xlsx', sheet_name='list')
#export.head(5)

### Query the database to count the number of samples

In [54]:
%%sql

SELECT DISTINCT MRN_NO, COUNT(MRN_NO) as NUM_SAMPLES FROM request WHERE no_sample = 0 GROUP BY MRN_NO;

 * sqlite:///list.db
Done.


MRN_NO,NUM_SAMPLES
M168867,4
M262597,3
M284302,2
M489804,4
M509175,4
M511995,5
M520426,2
M603017,4
M604071,4
M605081,4


### Query the database to create a new dataframe with num_samples

In [61]:
# Execute the SQL query
# cur.execute("SELECT DISTINCT MRN_NO, COUNT(MRN_NO) as NUM_SAMPLES FROM request WHERE no_sample = 0 GROUP BY MRN_NO;")
cur.execute("SELECT DISTINCT MRN_NO, COUNT(MRN_NO) as NUM_SAMPLES FROM request GROUP BY MRN_NO;")

# Fetch the results of the query
results = cur.fetchall()

# Get the column names from the cursor description
columns = [column[0] for column in cur.description]

# Convert the results into a pandas DataFrame
df = pd.DataFrame(results, columns=columns)

# Don't forget to close the connection when you're done
# conn.close()

df.rename(columns={'MRN_NO': 'MRN'}, inplace=True)
df.head(5)


,MRN,NUM_SAMPLES
0,M168867,5
1,M262597,3
2,M284302,4
3,M489804,4
4,M509175,4


### Create a for loop to loop through `export` and search every MRN in `request`

In [62]:
for i in range(len(export)):
    id = export.loc[i, 'MRN']

    row_number = df.index[df['MRN'].eq(id)]

    if not row_number.empty:
        export.loc[i, 'status'] = 'found'
        export.loc[i, 'num_samples'] = df.loc[row_number[0], 'NUM_SAMPLES']
    else:
        # print('MRN not found:', id)
        export.loc[i, 'status'] = 'not found'
        export.loc[i, 'num_samples'] = 'NULL'

print(len(export))

export.to_excel('result.xlsx', sheet_name='list', index=False)
# not_found.to_excel('not_found.xlsx', sheet_name='list', index=False)

229


In [63]:
conn.close()

## Mark the list in the plan
